**CCN-model**

Importamos las dependencias

In [ ]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

from torch import nn
from torch import Tensor
from PIL import Image
from torchvision.transforms import Compose, Resize, ToTensor
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torch.utils.data import random_split

Cargamos los datos

In [ ]:
transforms = Compose([Resize((224, 224)), ToTensor()])

training_data = ImageFolder(root="../input/iais22-birds/birds/birds", transform = ToTensor())
test_data = ImageFolder(root="../input/iais22-birds/submission_test", transform =  ToTensor())

train_set, test_set = random_split(training_data, (int(len(training_data) * 0.7) + 1, int(len(training_data) * 0.3)))

train_dataloader = DataLoader(train_set, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_set, batch_size=64, shuffle=True)

print(f"Training data size: {train_set}")

Creamos un diccionarios que mapea los ids de las clases con sus nombres

In [ ]:
clases_list = training_data.classes
clases = {}
cont = 0
for i in clases_list:
    clases[cont] = i
    cont+=1
print(clases)

Comprobamos que las imágenes están cargadas correctamente

In [ ]:
train_features, train_labels = training_data.__getitem__(0)
print(f"Tamaño de cada imagen: {train_features.size()}")
figure = plt.figure(figsize=(8, 8))
cols, rows = 3, 3
for i in range(1, cols * rows + 1):
    sample_idx = torch.randint(len(training_data), size=(1,)).item()
    img, label = training_data[sample_idx]
    figure.add_subplot(rows, cols, i)
    plt.title(clases[label])
    plt.axis("off")
    plt.imshow(img[1][:][:], cmap="gray")
plt.show()

Comprobamos que los DataLoaders funcionan correctamente

In [ ]:
for X, y in train_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Comprobamos si está disponible la GPU y la seleccionamos

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Creamos la arquitectura CNN basándonos en los módulos disponibles en Pytorch

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            #Input = 3 x 224 x 224, Output = 32 x 224 x 224
            nn.Conv2d(in_channels = 3, out_channels = 32, kernel_size = 3, padding = 1), 
            nn.ReLU(),
            nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 3, padding = 1), 
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),

            nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.Conv2d(in_channels = 128, out_channels = 256, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            
            nn.Conv2d(in_channels = 256, out_channels = 512, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            
            nn.Conv2d(in_channels = 512, out_channels = 1024, kernel_size = 3, padding = 1),
            nn.ReLU(),
            #Output = 1024 x 2 x 2
            nn.MaxPool2d(kernel_size=7),
  
            nn.Flatten(),
            nn.Linear(1024*4*4, 2048),
            nn.ReLU(),
            nn.Linear(2048, 1024),
            nn.ReLU(),
            nn.Linear(1024, 400)
        )
    
    def forward(self, image):
        image = self.network(image)
        return image

Definimos el modelo, seleccionamos la GPU para el entrenamiento y la funcion de perdida y de optimizacion

In [ ]:
model = CNN()
model = torch.load("./modelCNN75.pth")
model.to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=5e-3)
print(model)

Definimos las funciones de entrenamiento y testeo del modelo

In [ ]:
def train(train_dataloader, model, loss_fn, optimizer):
    size = len(train_dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(train_dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

Entrenamos y guardamos al modelo con distintos numero de epocas

In [ ]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")
torch.save(model, "modelCNN75.pth")
print("Model saved")

Creamos el fichero submission.csv para la evaluacion

In [ ]:
from os import remove
remove("submission.csv")
file_object = open('submission.csv', 'a')
file_object.write('Id,Category')
file_object.write("\n")
submision_dataloader = DataLoader(test_data, batch_size=1, shuffle=False)
model.eval()
id=0
with torch.no_grad():
    for x, y in submision_dataloader:  
        x, y = x.to(device), y.to(device)
        pred = model(x)
        file_object.write(f"{test_data.imgs[id]}"[56:].split(".")[0])
        file_object.write(",")
        file_object.write(f"{clases[pred.argmax(1).item()]}")
        file_object.write("\n")
        id=id+1
        
file_object.close()
print("Done!")

Comprobamos el número de parametros de nuestro modelo

In [ ]:
!pip install torchsummary

In [ ]:
from torchsummary import summary
summary(CNN(), (3, 224, 224), device='cpu')